## Instalación

In [ ]:
!pip install beautifulsoup4 
!pip install tqdm

## Configuración

In [1]:
from bs4 import BeautifulSoup
from requests import get

from tqdm.notebook import tqdm


year, month = 2024, 1
url = f'https://www.marbella.es/agenda/eventospormes/{year}/{month}.html'

### Peticiones web

In [2]:
response = get(url)
response

<Response [200]>

In [4]:
text = response.text

def preview_text(large_text: str) -> None:
    print(f'{len(large_text) = }')
    print(f'{large_text[:300]}\n\n...\n\n{large_text[-300:]}')

preview_text(text)

len(large_text) = 54321
<!DOCTYPE HTML>
<html prefix="og: http://ogp.me/ns#" lang="es-es" dir="ltr">
    <head>
        <meta charset="utf-8">
        <base href="https://www.marbella.es/agenda/crawler.listevents/-.html" />
	<meta http-equiv="content-type" content="text/html; charset=utf-8" />
	<meta name="keywords" conten

...

n-step-forward"></i>		</a>
	</li>
	<li>
		<a class="hasTooltip"  title="Final"  href="/agenda/crawler.listevents/-.html?start=8800">
			<i class="icon-forward"></i>		</a>
	</li>
		</ul>
	
			<input type="hidden" name="limitstart"
		       value="0"/>
	
</div>
		</form>
	</div>
	
    </body>
</html>


Podemos notar que hay referencias al endpoint `https://www.marbella.es/agenda/crawler.listevents/-.html?start=6900`, pero en este ejercicio lo vamos a ignorar.

Es posible que una request como la anterior no nos devuelva el mismo resultado que recibimos con una navegación normal. En ese caso, podemos intentar añadir headers que indiquen el navegador web que (teóricamente) estamos utilizando.

In [5]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/109.0'}

response = get(url, headers=headers)
response

<Response [200]>

In [6]:
text = response.text

preview_text(text)

len(large_text) = 770166
<!DOCTYPE html>
<html prefix="og: http://ogp.me/ns#" lang="es-es" dir="ltr">
    <head>
        <meta name="viewport" content="width=device-width, initial-scale=1">
        <link rel="icon" href="https://cdn.marbella.es/images/android-chrome-192x192.png" sizes="any">
                <link rel="icon"

...

lspy-class="uk-animation-slide-bottom"><a href="#" title="Volver al principio" uk-totop uk-scroll></a></div>
                
            
        
    
</div></div>
    </div>

    </div>


    </div>


</div>            </footer>
            
        </div>

        
        

    </body>
</html>


Para procesar el html recibido utilizamos BeautifulSoup

In [7]:
soup = BeautifulSoup(response.text)
# soup

Vamos a obtener todos los links que aparecen en la página

In [8]:
all_hrefs = [a['href'] for a in soup.find_all(href=True)]
len(all_hrefs)

577

Y filtramos por links que apunten a eventos

In [9]:
event_links = list({e for e in all_hrefs if 'eventodetalle' in e})
len(event_links)

265

In [10]:
event_links[-5:]

['/agenda/eventodetalle/58335/concurso-infantil-de-chirigotas.html',
 '/agenda/eventodetalle/58942/last-of-the-red-hot-lovers.html',
 '/agenda/eventodetalle/58948/torneo-beach-tenis-4-estaciones-invierno.html',
 '/agenda/eventodetalle/58912/si-esta-pa-ti-ni-aunque-te-quites.html',
 '/agenda/eventodetalle/58788/gestaciones-en-la-cuerda-felix-muyo.html']

Exploramos lo que aparece en una página de evento

In [11]:
base_url = f'https://www.marbella.es'

event = event_links[0]

html_e = get(f'{base_url}{event}', headers=headers)

soup = BeautifulSoup(html_e.text)
# soup

Una función para obtener la información de un evento a partir de su link:

In [12]:
from time import sleep


def get_event_info(event_link: str) -> dict:

    html = get(f'{base_url}{event_link}', headers=headers)
    text = html.text
    soup = BeautifulSoup(text)
    base = {'event_link': event_link}
    try:
        div_info = soup.find('div', string='Repetición Anterior').parent.parent.parent
    except:
        return base
    info = {
        'title': div_info.h1.text,
        'category': div_info.h2.text,
        'datetime': div_info.div.div.text.replace('\xa0', '').replace('\n', ''),
        'views': int(div_info.find(attrs={'class': 'hitslabel'}).parent.text.split(':')[-1].strip())
    }
    gmaps_dict = text.split('var gmapConf = {')[-1].split('}')[0]
    try:
        place = {
            'longitude': float(gmaps_dict.split('longitude:\t\'')[-1].split("'")[0]),
            'latitude': float(gmaps_dict.split('latitude:\t\'')[-1].split("'")[0]),
            'place_name': text.split('myEventDetailMapload(')[-1].split('.html?tmpl=component')[0].split(' \"/agenda/')[-1]
        }
    except:
        place = {}
    return {**base, **info, **place}


Recorremos las urls de los eventos que hemos obtenido obteniendo su información

In [13]:
events_info = []
for event_link in tqdm(event_links):
    events_info.append(get_event_info(event_link))
    sleep(.5)

len(events_info)

  0%|          | 0/265 [00:00<?, ?it/s]

265

Podemos poner la respuesta como una tabla

In [14]:
import pandas as pd


df = pd.DataFrame(events_info)
df

,event_link,title,category,datetime,views,longitude,latitude,place_name
0,/agenda/eventodetalle/58782/gestaciones-en-la-...,Gestaciones en la cuerda: Félix Muyo,"Exposiciones, Cultura y Enseñanza","Martes, 02 Enero 2024,10:00-19:00",10235.0,-4.889155,36.515846,lugares/detail/9/0/centro-cultural-cortijo-de-...
1,/agenda/eventodetalle/58786/gestaciones-en-la-...,Gestaciones en la cuerda: Félix Muyo,"Exposiciones, Cultura y Enseñanza","Lunes, 08 Enero 2024,10:00-19:00",10236.0,-4.889155,36.515846,lugares/detail/9/0/centro-cultural-cortijo-de-...
2,/agenda/eventodetalle/58329/concurso-infantil-...,Concurso infantil de Chirigotas,"Concursos, Fiestas","Lunes, 22 Enero 2024",15420.0,NaN,NaN,NaN
3,/agenda/eventodetalle/58470/mas-alla-de-los-su...,Más allá de los sueños: Mónica Vázquez Ayala,"Exposiciones, Cultura y Enseñanza","Martes, 30 Enero 2024,10:00",16789.0,-4.883007,36.510094,lugares/detail/1/0/museo-del-grabado-espanol-c...
4,/agenda/eventodetalle/58468/mas-alla-de-los-su...,Más allá de los sueños: Mónica Vázquez Ayala,"Exposiciones, Cultura y Enseñanza","Domingo, 28 Enero 2024,10:00",16790.0,-4.883007,36.510094,lugares/detail/1/0/museo-del-grabado-espanol-c...
...,...,...,...,...,...,...,...,...
260,/agenda/eventodetalle/58335/concurso-infantil-...,Concurso infantil de Chirigotas,"Concursos, Fiestas","Domingo, 28 Enero 2024",15450.0,NaN,NaN,NaN
261,/agenda/eventodetalle/58942/last-of-the-red-ho...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,/agenda/eventodetalle/58948/torneo-beach-tenis...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,/agenda/eventodetalle/58912/si-esta-pa-ti-ni-a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Función para descargar la información de todos los eventos de un mes

In [16]:
def get_all_month_events(year: int, month: int):

    month = str(month).rjust(2, '0')
    url = f'https://www.marbella.es/agenda/eventospormes/{year}/{month}.html'
    html = get(url, headers=headers)
    soup = BeautifulSoup(html.text)
    all_hrefs = [a['href'] for a in soup.find_all(href=True)]
    event_links = list({e for e in all_hrefs if 'eventodetalle' in e})

    events_info = []
    for event_link in tqdm(event_links):
        events_info.append(get_event_info(event_link))
        sleep(.5)
    df = pd.DataFrame(events_info)
    return df

df = get_all_month_events(2024, 1)
df

  0%|          | 0/265 [00:00<?, ?it/s]

,event_link,title,category,datetime,views,longitude,latitude,place_name
0,/agenda/eventodetalle/58782/gestaciones-en-la-...,Gestaciones en la cuerda: Félix Muyo,"Exposiciones, Cultura y Enseñanza","Martes, 02 Enero 2024,10:00-19:00",10260.0,-4.889155,36.515846,lugares/detail/9/0/centro-cultural-cortijo-de-...
1,/agenda/eventodetalle/58786/gestaciones-en-la-...,Gestaciones en la cuerda: Félix Muyo,"Exposiciones, Cultura y Enseñanza","Lunes, 08 Enero 2024,10:00-19:00",10261.0,-4.889155,36.515846,lugares/detail/9/0/centro-cultural-cortijo-de-...
2,/agenda/eventodetalle/58329/concurso-infantil-...,Concurso infantil de Chirigotas,"Concursos, Fiestas","Lunes, 22 Enero 2024",15451.0,NaN,NaN,NaN
3,/agenda/eventodetalle/58470/mas-alla-de-los-su...,Más allá de los sueños: Mónica Vázquez Ayala,"Exposiciones, Cultura y Enseñanza","Martes, 30 Enero 2024,10:00",16825.0,-4.883007,36.510094,lugares/detail/1/0/museo-del-grabado-espanol-c...
4,/agenda/eventodetalle/58468/mas-alla-de-los-su...,Más allá de los sueños: Mónica Vázquez Ayala,"Exposiciones, Cultura y Enseñanza","Domingo, 28 Enero 2024,10:00",16826.0,-4.883007,36.510094,lugares/detail/1/0/museo-del-grabado-espanol-c...
...,...,...,...,...,...,...,...,...
260,/agenda/eventodetalle/58335/concurso-infantil-...,Concurso infantil de Chirigotas,"Concursos, Fiestas","Domingo, 28 Enero 2024",15481.0,NaN,NaN,NaN
261,/agenda/eventodetalle/58942/last-of-the-red-ho...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,/agenda/eventodetalle/58948/torneo-beach-tenis...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,/agenda/eventodetalle/58912/si-esta-pa-ti-ni-a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Descarga de todos los eventos de 2024 y 2025

In [17]:
from itertools import product

dfs = []
for year, month in tqdm(product(range(2024, 2025), range(1, 13))):
    dfs.append(get_all_month_events(year, month))

df = pd.concat(dfs, ignore_index=True)
df

0it [00:00, ?it/s]

  0%|          | 0/265 [00:00<?, ?it/s]

  0%|          | 0/192 [00:00<?, ?it/s]

  0%|          | 0/130 [00:00<?, ?it/s]

  0%|          | 0/88 [00:00<?, ?it/s]

...

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

,event_link,title,category,datetime,views,longitude,latitude,place_name
0,/agenda/eventodetalle/58782/gestaciones-en-la-...,Gestaciones en la cuerda: Félix Muyo,"Exposiciones, Cultura y Enseñanza","Martes, 02 Enero 2024,10:00-19:00",10283.0,-4.889155,36.515846,lugares/detail/9/0/centro-cultural-cortijo-de-...
1,/agenda/eventodetalle/58786/gestaciones-en-la-...,Gestaciones en la cuerda: Félix Muyo,"Exposiciones, Cultura y Enseñanza","Lunes, 08 Enero 2024,10:00-19:00",10284.0,-4.889155,36.515846,lugares/detail/9/0/centro-cultural-cortijo-de-...
2,/agenda/eventodetalle/58329/concurso-infantil-...,Concurso infantil de Chirigotas,"Concursos, Fiestas","Lunes, 22 Enero 2024",15482.0,NaN,NaN,NaN
3,/agenda/eventodetalle/58470/mas-alla-de-los-su...,Más allá de los sueños: Mónica Vázquez Ayala,"Exposiciones, Cultura y Enseñanza","Martes, 30 Enero 2024,10:00",16857.0,-4.883007,36.510094,lugares/detail/1/0/museo-del-grabado-espanol-c...
4,/agenda/eventodetalle/58468/mas-alla-de-los-su...,Más allá de los sueños: Mónica Vázquez Ayala,"Exposiciones, Cultura y Enseñanza","Domingo, 28 Enero 2024,10:00",16858.0,-4.883007,36.510094,lugares/detail/1/0/museo-del-grabado-espanol-c...
...,...,...,...,...,...,...,...,...
1191,/agenda/eventodetalle/52174/villa-romana-de-ri...,Villa Romana de Río Verde,"Cultura y Enseñanza, Visita Guiada","Domingo, 15 Diciembre 2024,10:30-13:30",99768.0,-4.944436,36.495712,lugares/detail/2/0/villa-romana-de-rio-verde
1192,/agenda/eventodetalle/52180/villa-romana-de-ri...,Villa Romana de Río Verde,"Cultura y Enseñanza, Visita Guiada","Domingo, 29 Diciembre 2024,10:30-13:30",99769.0,-4.944436,36.495712,lugares/detail/2/0/villa-romana-de-rio-verde
1193,/agenda/eventodetalle/54315/basilica-paleocris...,Basílica Paleocristiana de Vega del Mar,"Cultura y Enseñanza, Tenencia Alcaldía San Ped...","Viernes, 06 Diciembre 2024,11:15-14:00",117096.0,-4.990243,36.471966,lugares/detail/27/0/basilica-paleocristiana-de...
1194,/agenda/eventodetalle/54317/basilica-paleocris...,Basílica Paleocristiana de Vega del Mar,"Cultura y Enseñanza, Tenencia Alcaldía San Ped...","Domingo, 08 Diciembre 2024,11:15-14:00",117097.0,-4.990243,36.471966,lugares/detail/27/0/basilica-paleocristiana-de...


In [19]:
df.to_csv('marbella_events_2024-2025.csv', index=False)